## Any Questions?

Student asked to go over `EXIST` VS `IN`

```sql
EXISTS(query Q)     -- true iff Q has non-empty result
```

vs

```sql
e IN (query Q)      -- true iff e is a member of Q's output
```

> Q: can we express `IN` using `EXISTS`?

```sql
SELECT  *           -- return the whole t for each t that satisfies the condition (homemade membership check using EXISTS)
FROM    T t, ...
WHERE   EXISTS (
    SELECT  *       -- doesn't matter since EXISTS only checks if result is non-empty
    FROM    S s
    WHERE   s.name == t.name
)
```

### Q: What about the JOIN primitive u can use in SQL?

A: You don't really need inner joins at all. Already addressed a long time ago just didn't name them. No need for special syntax. It's just FROM 2 relations with the WHERE clause has a condition checking for equality of a shared attribute. 

You don't get any additional expressive power using the inner join primitive. It's more of a taste/preference

EX: Find actors playing in **every** movie by "Berto"

```sql
SELECT  Actor   FROM Movie
WHERE   Actor   NOT IN (

    SELECT  m1.Actor
    FROM    Movie m1, Movie m2
    WHERE   m2.Director="Berto"
        AND m1.Actor NOT IN (
            SELECT  Actor
            FROM    Movie
            WHERE   Title=m2.Title
        )

)
```

Q: What does the doubly nested query compute?

A: Extracts the class for the movie m2 directed by "Berto"

So we basically find actors for which there is some movie by "Berto" in which they do not act to generate a list of violating actors.

Then we use the list of violating actors, and find just the valid actors instead using `NOT IN`

> SQL's way of saying this: 
 
"find the actors for which there is no movie by Bertolucci in which they do not act"

OR equivalently

"find the actors not among those actors for which there is some movie by Bertolucci in which they do not act

## Example of EXIST

EX: Find titles of curr playing movies by Berto

```sql
SELECT  s.title
FROM    schedule s
WHERE   EXISTS (
    SELECT  *                           -- Again, doesn't matter for EXISTS
    FROM    movie
    WHERE   movie.title = s.title 
        AND movie.director = "Berto"
)
```

Correlation used

### Boolean Prediate

useful for constraints, e.g. "everybody likes UCSD"

Person(name: string)

Likes(name: string, school: string)

```sql
NOT EXISTS
    (
        SELECT  *   FROM PERSON
        WHERE   NOT EXISTS (
            SELECT  *   FROM LIKES
            WHERE   PERSON.name = LIKES.name
                AND school='UCSD'
        )
    )
```

# Union, Intersection & Difference

Usually for chaining operatoins over tables. This all behind relational algebra: the way in which the plan for answering the query is presented internally in the DBMS.

- UNION: `<SQL Query 1> UNION <SQL Query 2>`

- Intersection: `<SQL Query 1> INTERSECT <SQL Query 2>`

- EXCEPT: `<SQL Query 1> EXCEPT <SQL Query 2>`

Set operations: might return multisets (contain duplicates)

### Example

Find all actors or directors

```sql
(SELECT Actor AS Name
 FROM   Movie)

UNION

(SELECT Director AS Name
 FROM   Movie)
```

> Note: AS is required!
 
> Union Compatability: Schema should be specified so that the operator knows how to perform the union operator on the tuples/objects within

### Example

Find all actors who are NOT directors

```sql
(SELECT Actor AS Name
 FROM   Movie)

EXCEPT

(SELECT Director AS Name
 FROM   Movie)
```


## Multiplicity-Aware Operations

> Just add the `ALL` keyword

- `UNION ALL`
- `INTERSECT ALL`
- `EXCEPT ALL`

#### Basic interpretation:

`{..x^3..} UNION ALL {..x^2..} => {..x^5..}`

`{..x^3..} INTERSECT ALL {..x^2..} => {..x^2..}`

`{..x^3..} EXCEPT ALL {..x^2..} => ..x..`

#### Additional Case

`{..x^2..} EXCEPT ALL {..x^3..} => no x`

Takes the max(A-B, 0)

## Natural [inner] Join

- Combines tuples from 2 tables by matching on common attributes

![](images/2022-10-25-15-13-45.png)

Notice
- Movie's "Tango" tuple joins w/ both of schedule's "Tango" tuples
- Movie's "Psycho" tuple joins w/ Schedule's "Psycho" tuple
- Movie's "Sky" tuple and Schedule's "Bambi" tuples have no matches and are EXCLUDED from result

EX: Find the directors of all movies showing in Hilcrest

```sql
SELECT  director
FROM    movie natural join schedule
WHERE   theaer = "Hillcrest"
```

**Question**: Can we write this in a different way?

```sql
SELECT	director
FROM    from movie, schedule
WHERE	movie.title = schedule.title
    AND theater = "Hillcrest"
```

> Note: More variations of joins are available in SQL...